In [1]:
import numpy as np
import scipy
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras import backend as K
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold  # If needed


In [2]:
PATH = os.getcwd()
print (PATH)

D:\capstone\Classification-of-Hyperspectral-Image-master


In [3]:
# Load the Global values (windowSize, numPCAcomponents, testRatio) from the text file global_variables.txt
myFile = open('global_variables.txt', 'r') 
file = myFile.readlines()[:]


for line in file:

    if line[0:3] == "win":

        ds = line.find('=')
        windowSize = int(line[ds+1:-1],10)

    elif line[0:3] == "num":

        ds = line.find('=')
        numPCAcomponents = int(line[ds+2:-1],10)

    else:

        ds = line.find('=')
        testRatio = float(line[ds+1:])


In [4]:
# Global Variables
#windowSize = 5
#numPCAcomponents = 30
#testRatio = 0.25

# Load Training Dataset

In [5]:
X_train = np.load("X_trainPatches_" + str(windowSize) + "PCA" + str(numPCAcomponents) + "testRatio" + str(testRatio)  + ".npy")

y_train = np.load("y_trainPatches_" + str(windowSize) + "PCA" + str(numPCAcomponents) + "testRatio" + str(testRatio) + ".npy")

In [6]:
# Reshape into (numberofsamples, channels, height, width)
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[3], X_train.shape[1], X_train.shape[2]))

In [7]:
y_train = to_categorical(y_train)

In [8]:
# Define the input shape 
input_shape= X_train[0].shape
print(input_shape)

(30, 5, 5)


In [9]:
# number of filters
C1 = 3*numPCAcomponents

In [10]:
# Define the model
model = Sequential()

model.add(Conv2D(C1, (3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(3*C1, (3, 3), activation='relu'))
model.add(Dropout(0.25))



model.add(Flatten())
model.add(Dense(6*numPCAcomponents, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='softmax'))

D:\capstone\Classification-of-Hyperspectral-Image-master\your_env_name\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
from tensorflow.keras.optimizers import SGD

# Update the SGD optimizer with correct parameters
sgd = SGD(learning_rate=0.0001, momentum=0.9, nesterov=True, weight_decay=1e-6)

# Compile your model
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


In [12]:
model.fit(X_train, y_train, batch_size=32, epochs=15)

Epoch 1/15
928/928 ━━━━━━━━━━━━━━━━━━━━ 48s 51ms/step - accuracy: 0.1824 - loss: 2.6119
Epoch 2/15
928/928 ━━━━━━━━━━━━━━━━━━━━ 49s 53ms/step - accuracy: 0.5290 - loss: 1.7291
Epoch 3/15
928/928 ━━━━━━━━━━━━━━━━━━━━ 50s 54ms/step - accuracy: 0.6475 - loss: 1.1487
Epoch 4/15
928/928 ━━━━━━━━━━━━━━━━━━━━ 48s 52ms/step - accuracy: 0.7195 - loss: 0.8846
Epoch 5/15
928/928 ━━━━━━━━━━━━━━━━━━━━ 84s 54ms/step - accuracy: 0.7579 - loss: 0.7339
Epoch 6/15
928/928 ━━━━━━━━━━━━━━━━━━━━ 82s 53ms/step - accuracy: 0.7824 - loss: 0.6537
Epoch 7/15
928/928 ━━━━━━━━━━━━━━━━━━━━ 82s 53ms/step - accuracy: 0.8016 - loss: 0.5924
Epoch 8/15
928/928 ━━━━━━━━━━━━━━━━━━━━ 80s 50ms/step - accuracy: 0.8252 - loss: 0.5286
Epoch 9/15
928/928 ━━━━━━━━━━━━━━━━━━━━ 49s 52ms/step - accuracy: 0.8460 - loss: 0.4720
Epoch 10/15
928/928 ━━━━━━━━━━━━━━━━━━━━ 80s 50ms/step - accuracy: 0.8531 - loss: 0.4440
Epoch 11/15
928/928 ━━━━━━━━━━━━━━━━━━━━ 83s 51ms/step - accuracy: 0.8625 - loss: 0.4219
Epoch 12/15
928/928 ━━━━━━━━━━

In [13]:
import h5py
from keras.models import load_model

In [14]:
model.save('my_model' + str(windowSize) + 'PCA' + str(numPCAcomponents) + "testRatio" + str(testRatio) + '.h5')